In [29]:
import warnings
# run block of code and catch warnings
with warnings.catch_warnings():
 # ignore all caught warnings
    warnings.filterwarnings("ignore")
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score
    from sklearn.tree import DecisionTreeClassifier


In [30]:
    warnings.filterwarnings("ignore")
    data = pd.read_csv("Face-Feat.txt")
    print(data)
    labels = data["Emotion"]
    inputs = data.drop("Emotion", axis = 1)
    data_in, test_in, data_out, test_out = train_test_split(inputs, labels, test_size=0.1, stratify=labels)
    train_in, val_in, train_out, val_out = train_test_split(data_in, data_out, test_size = 0.2/0.9, stratify = data_out)

     Emotion         1         2         3         4         5         6  \
0   NEGATIVE  0.915262  0.744733  0.146054  0.144566  0.461864  0.604585   
1   NEGATIVE  0.884081  0.396989  0.169706  0.142127  0.656049  0.600000   
2   NEGATIVE  0.856672  0.305868  0.122347  0.134102  0.449069  0.693600   
3   NEGATIVE  0.773486  0.209016  0.140429  0.127732  0.452869  0.592668   
4   NEGATIVE  0.830810  0.374441  0.159301  0.155777  0.566022  0.637202   
..       ...       ...       ...       ...       ...       ...       ...   
63   NEUTRAL  0.864904  0.531306  0.143059  0.150276  0.611580  0.564786   
64   NEUTRAL  0.781025  0.240832  0.092195  0.067082  0.676757  0.588218   
65   NEUTRAL  0.817953  0.274721  0.045164  0.045164  0.594038  0.678962   
66   NEUTRAL  0.731220  0.268802  0.038014  0.038014  0.462464  0.618828   
67   NEUTRAL  0.687594  0.234346  0.133712  0.127163  0.624924  0.609501   

           7         8         9  ...        13        14        15        16  \
0   0.

In [31]:
    model = SVC()
    model.fit(train_in, train_out)
    output = model.predict(val_in)
    print("Val accuracy is ", accuracy_score(val_out, output)*100)

Val accuracy is  78.57142857142857


In [34]:
    model_2 = DecisionTreeClassifier()
    model_2.fit(train_in, train_out)
    output = model_2.predict(val_in)
    print("Val accuracy is ", accuracy_score(val_out, output)*100)

Val accuracy is  64.28571428571429


In [38]:
    param_grid=[
        {"kernel": ["linear","poly","rbf"]},
        {"kernel": ["poly"], "degree": [2, 5, 15]}
    ]
    meta_model = GridSearchCV(SVC(), param_grid=param_grid)
    meta_model.fit(train_in, train_out)
    output = meta_model.predict(test_in)
    print("Val accuracy is ", accuracy_score(test_out, output)*100)
    print(meta_model.best_params_)

Val accuracy is  71.42857142857143
{'kernel': 'poly'}


In [16]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from torch import nn

In [17]:
class MyNeuralNetwork(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.net(nn.Sequential(
            nn.Linear(22, 100),
            nn.ReLU(),
            nn.Linear(100, 3)
        ))
    
    def forward(self, inputs):
        return self.net(inputs)

        

In [18]:
class MultiEmoVA(Dataset):
    def __init__(self, data_path) -> None:
        super().__init__()
        data=pd.read_csv("Face-Feat.txt")
        self.inputs = torch.tensor(data.drop("Emotion", axis=1).to_numpy(np.float32))
        self.index2label = [label for label in data["Emotion"].unique()]
        label2index = {label: i for i, label in enumerate(self.index2label)}
        self.labels = torch.tensor(data["Emotion"].apply(lambda x: label2index[x]))
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]


In [19]:
data = MultiEmoVA("Face-Feat.txt")
#print(data[0])
train, val, test = random_split(data, [0.7, 0.2, 0.1])

model = MyNeuralNetwork()
#print(model(train[0][0]).softmax(dim=0).argmax(dim=0))

train_loader = DataLoader(train)

loss_fn = nn.CrossEntropyLoss()

optim = torch.optim.ADAM(model.parameters(), lr = 0.01)

for inputs, labels in train_loader:
    output = model(inputs)

    loss = loss_fn(output, labels)
    #print(loss)

    loss.backward()
    optim.step()
    optim.zero_grad()

test_loader = DataLoader(test)

for inputs, labels, in test_loader:
    output = model(inputs)
    crrect += (ouput.softmax(dim=0).argmax(dim=1) == labels).sum()
    



AttributeError: 'MyNeuralNetwork' object has no attribute 'net'